In [1]:
import pyomo.environ as pe # Pyomo environment
from idaes.core import FlowsheetBlock, StateBlock
from idaes.models.unit_models import HeatExchanger
from idaes.models.unit_models.heat_exchanger import HX0DInitializer
from idaes.models.unit_models.heat_exchanger import delta_temperature_amtd_callback
from idaes.models.properties import iapws95

In [2]:
# Create an empty flowsheet and steam property parameter block.
model = pe.ConcreteModel()
model.fs = FlowsheetBlock(dynamic=False)
model.fs.properties = iapws95.Iapws95ParameterBlock()

In [3]:
# Add a hx model to the flowsheet.
model.fs.heat_exchanger = HeatExchanger(
    delta_temperature_callback=delta_temperature_amtd_callback,
    hot_side_name="shell",
    cold_side_name="tube",
    shell={"property_package": model.fs.properties},
    tube={"property_package": model.fs.properties}
)

In [4]:
# Load the area variable that was stored in HE_AreaCalculation.ipynb
%store -r A

# Now you can use the area variable
print(A)

4.508128193938703


In [5]:
model.fs.heat_exchanger.area.fix(A)
model.fs.heat_exchanger.overall_heat_transfer_coefficient[0].fix(100)
model.fs.heat_exchanger.shell_inlet.flow_mol.fix(100)
model.fs.heat_exchanger.shell_inlet.pressure.fix(101325)
model.fs.heat_exchanger.shell_inlet.enth_mol.fix(4000)
model.fs.heat_exchanger.tube_inlet.flow_mol.fix(100)
model.fs.heat_exchanger.tube_inlet.pressure.fix(101325)
model.fs.heat_exchanger.tube_inlet.enth_mol.fix(3000)

Initialize the model

In [6]:

# Initialize the model
initializer = HX0DInitializer()
initializer.initialize(model.fs.heat_exchanger)

2025-05-12 15:53:58 [INFO] idaes.init.fs.heat_exchanger: Initialization Completed, optimal - <undefined>


<InitializationStatus.Ok: 1>

In [7]:
from idaes.core.util.model_statistics import degrees_of_freedom
print("Degrees of Freedom:", degrees_of_freedom(model))


Degrees of Freedom: 0


Solve the model

In [8]:
solver = pe.SolverFactory("ipopt")
results = solver.solve(model, tee=True)


Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [9]:
area_value = model.fs.heat_exchanger.area()

# Print out the area of the heat exchanger
print(f"Heat Exchanger Area: {area_value} m²")

Heat Exchanger Area: 4.508128193938703 m²


In [10]:
import idaes_ui

In [11]:
import idaes_ui.fv
dir(idaes_ui.fv)

['VisualizeResult',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'errors',
 'flowsheet',
 'fsvis',
 'icons',
 'model_server',
 'models',
 'persist',
 'validate_flowsheet',
 'visualize']

In [12]:
from idaes_ui.fv import visualize 

In [13]:
visualize(model.fs, "HeatExchanger")

2025-05-12 15:54:12 [INFO] idaes.idaes_ui.fv.fsvis: Started visualization server
2025-05-12 15:54:12 [INFO] idaes.idaes_ui.fv.fsvis: Loading saved flowsheet from 'HeatExchanger.json'
2025-05-12 15:54:12 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'HeatExchanger.json' in current directory (c:\Users\Sara\Desktop\IDAES\HE)
2025-05-12 15:54:12 [INFO] idaes.idaes_ui.fv.fsvis: Flowsheet visualization at: http://localhost:58346/app?id=HeatExchanger


VisualizeResult(store=<idaes_ui.fv.persist.FileDataStore object at 0x000001CD6A64CFD0>, port=58346, server=<idaes_ui.fv.model_server.FlowsheetServer object at 0x000001CD6A64D2A0>)